In [1]:
!pip install pydantic[email]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [2]:
import json
import pandas as pd
from pydantic import BaseModel, EmailStr, validator, ValidationError

[Отличный Гайд по pydantic для DS-проектов](https://pub.towardsai.net/data-reliability-101-a-practical-guide-to-data-validation-using-pydantic-in-data-science-projects-3392968c3646#acff)

## Создадим протую модель для валидации карточек сотрудников

In [8]:
class Employee(BaseModel):
  name: str
  age: int
  email: EmailStr
  department: str
  employee_id: str

  @validator("employee_id")
  def validate_employee_id(cls, v):
      if not v.isalnum() or len(v) != 6:
          raise ValueError("Employee ID must be exactly 6 alphanumeric characters")
      return v

  @validator("age")
  def validate_age(cls, v):
      if not (18 <= v <= 65):
          raise ValueError("Age must be between 18 and 65")
      return v

/tmp/ipykernel_48598/2303925220.py:8: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("employee_id")
/tmp/ipykernel_48598/2303925220.py:14: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("age")


## Тестирование валидации возраста

In [9]:
# Тестируем валидацию возраста
test_cases = [
    {"name": "John Doe", "age": 30, "email": "john@example.com", "department": "IT", "employee_id": "ABC123"},  # Корректный возраст
    {"name": "Jane Smith", "age": 17, "email": "jane@example.com", "department": "HR", "employee_id": "XYZ789"},  # Слишком молодой
    {"name": "Bob Johnson", "age": 70, "email": "bob@example.com", "department": "Finance", "employee_id": "DEF456"},  # Слишком старый
    {"name": "Alice Brown", "age": 25, "email": "alice@example.com", "department": "Marketing", "employee_id": "GHI789"},  # Корректный возраст
]

print("Тестирование валидации возраста:")
print("=" * 50)

for i, record in enumerate(test_cases, 1):
    try:
        employee = Employee(**record)
        print(f"✅ Тест {i}: Валидный сотрудник - {employee.name} (возраст: {employee.age})")
    except ValidationError as e:
        print(f"❌ Тест {i}: Невалидный сотрудник - {record['name']} (возраст: {record['age']})")
        for error in e.errors():
            print(f"   Ошибка: {error['msg']}")

Тестирование валидации возраста:
✅ Тест 1: Валидный сотрудник - John Doe (возраст: 30)
❌ Тест 2: Невалидный сотрудник - Jane Smith (возраст: 17)
   Ошибка: Value error, Age must be between 18 and 65
❌ Тест 3: Невалидный сотрудник - Bob Johnson (возраст: 70)
   Ошибка: Value error, Age must be between 18 and 65
✅ Тест 4: Валидный сотрудник - Alice Brown (возраст: 25)


### Объяснение валидации возраста

Валидатор `@validator("age")` проверяет, что возраст сотрудника находится в диапазоне от 18 до 65 лет включительно. Это предотвращает создание записей с нереалистичными значениями возраста.

**Логика валидации:**
- Минимальный возраст: 18 лет (сотрудники должны быть совершеннолетними)
- Максимальный возраст: 65 лет (типичный пенсионный возраст)

Если возраст выходит за эти границы, будет выброшено исключение `ValidationError` с понятным сообщением об ошибке.

In [11]:
with open("employees.json", "r") as f:
    data = json.load(f)

# проверить каждого сотрудника
for record in data:
    try:
        employee = Employee(**record)
        print(f"Valid employee record: {employee.name}")
    except ValidationError as e:
        print(f"Invalid employee record: {record['name']}")
        print(f"Errors: {e.errors()}")

Valid employee record: John Doe
Invalid employee record: Jane Smith
Errors: [{'type': 'value_error', 'loc': ('age',), 'msg': 'Value error, Age must be between 18 and 65', 'input': 99, 'ctx': {'error': ValueError('Age must be between 18 and 65')}, 'url': 'https://errors.pydantic.dev/2.11/v/value_error'}]
Valid employee record: Alice Brown
Valid employee record: Dave West
